In [6]:
# Import
import itertools
import numpy as np

from munkres import Munkres

In [7]:
def get_sols(r):
    n=int(r.readline())
    for _ in range(n):
        sols.append(list(map(int, r.readline().split()))[1:])
    return sols

In [8]:
def load_datas(filename: object, N_obj: object, get_sol=True) -> object:
    objectifs = []
    with open(filename, 'r') as r:
        dim = int(r.readline())
        for _ in range(4):
            tmp = [list(map(int, r.readline().split())) for _ in range(dim)]
            objectifs.append(np.array(tmp))

        sols=[]
        if get_sol:
            sols = get_sols(r)
    return objectifs[:N_obj], dim, sols

In [9]:
def compute_objectif(objectifs, x, dim):
    return [int(sum(sum(x*objectifs[i]))) for i in range(len(objectifs))]

In [10]:
def random_permute(x,N_permute):
    # return [np.random.permutation(x) for _ in range(N_permute)]
    return [np.random.permutation(x) for _ in range(N_permute)]+ [x[:, np.random.permutation(x.shape[1])] for _ in range(N_permute)]

In [11]:
def mario_kart(x, dim):
    # rol = []
    # rol2=[]
    # for i in range(dim//5):
    #     rol.append(np.roll(x, i, axis=0))
    #     rol2.append(np.roll(x, i, axis=1))
    #     rol.append(np.roll(x, -i, axis=0))
    #     rol2.append(np.roll(x, -i, axis=1))

    honey_row_last = np.roll(x, 1, axis=0)
    honey_row_first= np.roll(x, -1, axis=0)
    honey_col_first= np.roll(x, 1, axis=1)
    honey_col_last= np.roll(x, -1, axis=1)
    # honey_col_last1 = np.array([[x[j][-1],x[j][:-1]] for j in ran ge(dim)])
    # honey_col_first1 = np.array([[x[j][1:],x[j][0]] for j in range(dim)])
    # print(honey_col_first), print(honey_col_last),  print(x)
    return [honey_row_first, honey_row_last, honey_col_first, honey_col_last]
    # return [honey_row_first]
    # return rol+rol2



In [12]:
def calculResult(alldata,newSolution):
    vecResult = []
    for k in range(len(alldata)):
        result = 0
        for i in range(len(newSolution)):
            for j in range(len(newSolution[i])):
                result = result+ newSolution[i][j]*alldata[k][i][j]
        vecResult.append(result)
    return vecResult

In [13]:
def update(A, Pa, obj_voisin, voisin, N_obj):
    bad_memory=[]
    O_O = False
    global_counter=0
    for i in range(len(A)):
        A_grand=0
        A_petit=0
        for k in range(N_obj):
            if A[i][0][k]>=obj_voisin[k]:
                A_grand+=1
            if A[i][0][k]<=obj_voisin[k]:
                A_petit+=1
        if A_petit == N_obj:
            O_O = False
            break
        if A_grand == N_obj:
            bad_memory.append(i)
            good_memory = obj_voisin
            O_O = True
        global_counter+=1
    for indice in sorted(bad_memory, reverse=True):
        A.pop(indice)
    if global_counter == len(A)-len(bad_memory):
        Pa.append(voisin)
        A.append((obj_voisin, voisin))


    return A, Pa

In [14]:
N_obj= 4
lap=15
objectifs, dim, sols = load_datas(f'datas/LAP-{lap}.dat', N_obj, get_sol=False)
print(objectifs)

[array([[ 1,  6,  5,  2,  5, 10,  3,  6,  7,  8,  9,  2,  7,  2,  9],
       [ 2,  7, 10,  3, 10,  3,  6,  3,  2,  9,  6,  5,  2,  7,  6],
       [ 3,  2,  3,  6,  1,  6,  3, 10,  5,  6,  7, 10,  5,  4,  5],
       [ 2,  3,  6,  5,  8,  3,  8,  7,  8,  9,  6,  3,  4,  1,  6],
       [ 5,  6,  7,  4,  3, 10,  7,  2,  5,  4,  7,  2,  9, 10,  5],
       [ 4,  1,  2,  9, 10,  9,  4,  7,  2,  3,  8,  3,  6,  3,  6],
       [ 5,  4,  9,  6,  3,  6,  9, 10,  7,  6,  9,  6,  7, 10,  3],
       [ 2,  7, 10,  1,  8,  3,  6,  9,  6,  1,  8,  3, 10,  1, 10],
       [ 7,  6,  1,  8,  9,  8,  7,  4,  1,  6,  3,  2,  1,  8,  3],
       [ 8,  9,  2,  1, 10,  3,  4,  7,  4,  9,  4,  9,  2,  3,  4],
       [ 5,  2,  5,  4,  1,  4,  9,  2,  1,  2,  1, 10,  1,  6,  3],
       [10,  9,  8,  1,  6,  1,  6,  7,  6,  9,  6,  3,  4,  9,  8],
       [ 9,  4,  5,  4,  5,  6,  9,  4,  3,  8,  9, 10,  1,  6,  1],
       [ 2,  7,  2,  7,  6,  1,  6,  5,  6,  1,  6,  5,  6,  1,  8],
       [ 5,  4,  9,  2,  5,  6,  

In [15]:
obj_ref = np.copy(objectifs)
mean_obj = np.zeros((dim,dim))
for obj in obj_ref:
    mean_obj = np.add(mean_obj, obj)
mean_obj=mean_obj/N_obj
b = np.zeros_like(mean_obj)
b[np.arange(len(mean_obj)), mean_obj.argmax(1)] = 1
c = np.zeros_like(mean_obj)
c[np.arange(len(mean_obj)), mean_obj.argmin(1)] = -1
d = np.zeros_like(mean_obj)
d[np.arange(len(mean_obj)), mean_obj.argmax(0)] = 1
e = np.zeros_like(mean_obj)
e[np.arange(len(mean_obj)), mean_obj.argmin(0)] = -1
print(f"b: {b}")
print(f"c: {c}")
print(f"d: {d}")
print(f"e: {e}")
i1 = np.add(b,c)
i2 = np.add(d,e)
print(f"i1: {i1}")
print(f"i2: {i2}")
print(np.add(i1, i2))

b: [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
c: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.

In [16]:
print(mean_obj.argmin(0))

[12  5  0  6 12  1 12  1  8 10  8  4  6  0  4]


In [17]:
start = np.identity(dim)
flip_start = np.fliplr(start)

In [18]:
vals =[]
objs_copy = np.copy(objectifs)
for i in range(len(objectifs)):
    tmp = np.zeros((dim, dim))
    m = Munkres()
    indexes = m.compute(objs_copy[i])
    print(indexes)
    for index in indexes:
        tmp[index[0]][index[1]]=1
    vals.append(tmp)
    sol = compute_objectif(objectifs, tmp, dim)

[(0, 6), (1, 0), (2, 4), (3, 13), (4, 7), (5, 1), (6, 14), (7, 3), (8, 11), (9, 2), (10, 8), (11, 5), (12, 12), (13, 9), (14, 10)]
[(0, 13), (1, 0), (2, 3), (3, 8), (4, 5), (5, 6), (6, 7), (7, 1), (8, 2), (9, 14), (10, 11), (11, 9), (12, 10), (13, 12), (14, 4)]
[(0, 9), (1, 5), (2, 3), (3, 1), (4, 14), (5, 10), (6, 12), (7, 7), (8, 8), (9, 13), (10, 6), (11, 0), (12, 11), (13, 4), (14, 2)]
[(0, 3), (1, 5), (2, 14), (3, 6), (4, 11), (5, 1), (6, 10), (7, 9), (8, 8), (9, 2), (10, 4), (11, 12), (12, 0), (13, 13), (14, 7)]


In [19]:
P_temp =vals+ [start, flip_start]
P_temp+=random_permute(P_temp[0], 10)
P_temp+=random_permute(P_temp[1], 10)
P_temp+=random_permute(P_temp[2], 10)
tmp=[]
for p in P_temp:
    tmp+=random_permute(p, 10)
P_temp+=tmp
for t in tmp:
    P_temp+=mario_kart(t, dim)
tmp=[]
for p in P_temp:
    tmp+=random_permute(p, 10)
P_temp+=tmp
A = [] # archive val obj
Pa=[]
for p in P_temp:
    obj_voisin = compute_objectif(objectifs, p, dim)
    if len(A)==0:
        A.append((obj_voisin, p))
        Pa.append(p)
        pass
    A, Pa = update(A, Pa, obj_voisin, p, N_obj)
P=Pa
Pa = []
# print(f"Lenght of A: {len(A)}")
# print(f"Lenght of P: {len(P)}")
# for a in A:
#     print(a[0])
# A = []
N = 1

In [20]:
import time

start = time.time()
stopper = 0
while P:
    break
    i_start = time.time()
    print(f"Iterations: {stopper+1}")
    print(f"Lenght of A: {len(A)}")
    print(f"Lenght of P: {len(P)}")
    for x in P:
        voisinage = mario_kart(x,dim)
        tmp=[]
        for voisin in voisinage:
            tmp+=random_permute(voisin, 10)
        # voisinage2 = []
        # for t in tmp:
        #     voisinage2+=mario_kart(t, dim)
        voisinage+=tmp
        voisinage+=random_permute(x, 100)
        # voisinage+=voisinage2
        for voisin in voisinage:
            obj_voisin = compute_objectif(objectifs, voisin, dim)
            A, Pa = update(A, Pa, obj_voisin, voisin, N_obj)
    i_end = time.time()
    print(f"Iteration time: {i_end-i_start}")
    print("----------------------------------")
    P = Pa
    Pa = []
    stopper+=1
end = time.time()

In [21]:
erreurs = 0

for a in A:
    if a[0] not in sols:
        erreurs+=1
        print(a[0])

print(f"Erreurs: {erreurs}/{len(A)}")
# print(sols)

[23, 172, 197, 282]
[93, 36, 201, 326]
[97, 160, 31, 286]
[55, 180, 173, 62]
[57, 84, 245, 294]
[37, 168, 201, 274]
[83, 148, 89, 270]
[67, 132, 263, 170]
[41, 128, 207, 326]
[61, 132, 229, 186]
[83, 152, 117, 218]
[59, 176, 129, 242]
[79, 68, 233, 346]
[73, 168, 121, 206]
[51, 108, 207, 314]
[95, 148, 93, 226]
[55, 104, 205, 290]
[43, 192, 213, 254]
[49, 116, 211, 290]
[53, 108, 211, 306]
[85, 172, 119, 154]
[89, 76, 231, 286]
[65, 156, 229, 154]
[53, 108, 181, 366]
[89, 116, 153, 210]
[75, 124, 167, 186]
[83, 128, 181, 166]
[45, 188, 183, 250]
[79, 148, 245, 126]
[57, 168, 199, 154]
[37, 148, 197, 306]
[85, 128, 169, 162]
[47, 164, 181, 258]
[91, 76, 227, 226]
[83, 124, 117, 250]
[55, 100, 223, 330]
[53, 168, 237, 190]
[63, 164, 131, 258]
[57, 132, 197, 226]
[73, 92, 233, 210]
[55, 152, 221, 194]
[67, 164, 183, 146]
[65, 168, 161, 162]
[77, 108, 209, 198]
[47, 152, 199, 258]
[59, 144, 251, 186]
[71, 152, 135, 210]
[67, 124, 209, 186]
[47, 124, 229, 282]
[51, 156, 139, 310]
[73, 132, 

In [22]:
# from datetime import timedelta
# with open(f'sols_Lap-{lap}_{timedelta(seconds=end-start)}.txt', 'w') as w:
#     w.write(f'{len(A)}\n')
#     for a in A:
#         # print(*a[0])
#         w.write(' '.join(str(p) for p in a[0]))
#         w.write('\n')

In [23]:
from pymoo.indicators.hv import HV
pareto=[]
for a in A:
    pareto.append(np.array(a[0]))
print(pareto)
pareto=np.array(pareto)
print(pareto)
ref_15 = np.array([112, 206, 247, 418])
ref_20=np.array([137, 304, 414, 531])
ref_30 = np.array([211,381,581,835])
if lap ==15:
    ref_point = ref_15
elif lap ==20:
    ref_point=ref_20
elif lap ==30:
    ref_point=ref_30
else:
    pt=900
    ref_point = np.array([pt, pt, pt, pt])

ind = HV(ref_point=ref_point)
hv = ind(pareto)
print("HV", hv)
print(f"Norm: {hv/(np.prod(ref_point))}")

[array([ 23, 172, 197, 282]), array([ 93,  36, 201, 326]), array([ 97, 160,  31, 286]), array([ 55, 180, 173,  62]), array([ 57,  84, 245, 294]), array([ 37, 168, 201, 274]), array([ 83, 148,  89, 270]), array([ 67, 132, 263, 170]), array([ 41, 128, 207, 326]), array([ 61, 132, 229, 186]), array([ 83, 152, 117, 218]), array([ 59, 176, 129, 242]), array([ 79,  68, 233, 346]), array([ 73, 168, 121, 206]), array([ 51, 108, 207, 314]), array([ 95, 148,  93, 226]), array([ 55, 104, 205, 290]), array([ 43, 192, 213, 254]), array([ 49, 116, 211, 290]), array([ 53, 108, 211, 306]), array([ 85, 172, 119, 154]), array([ 89,  76, 231, 286]), array([ 65, 156, 229, 154]), array([ 53, 108, 181, 366]), array([ 89, 116, 153, 210]), array([ 75, 124, 167, 186]), array([ 83, 128, 181, 166]), array([ 45, 188, 183, 250]), array([ 79, 148, 245, 126]), array([ 57, 168, 199, 154]), array([ 37, 148, 197, 306]), array([ 85, 128, 169, 162]), array([ 47, 164, 181, 258]), array([ 91,  76, 227, 226]), array([ 83, 1

In [24]:
P = [start, flip_start]
Pa = []
A = [] # archieve val obj
N = 1
for x in P:
    A_temp = []
    for k in range(N_obj):
        A_temp.append(compute_objectif(objectifs[k], x, dim))

    A.append((x,A_temp))



tuple_karts=[]
for x in P:
    fxs = []
    for k in range(N_obj):
        fxs.append(compute_objectif(objectifs[k], x, dim))
    karts = mario_kart(x,dim)


    for kart in karts:
        kart_eval = []
        for k in range(N_obj):
            kart_eval.append(compute_objectif(objectifs[k], kart, dim))
        tuple_karts.append((kart, kart_eval))


good_memory= {}
bad_memory= set()
for i in range(len(A)):
    good=0
    bad=0
    for k in range(N_obj):
        if A[i][1][k]>tuple_karts[1][]:
            bad-=1
        elif A[i][k]<kart_eval[k]:
            good+=1
    if good == N_obj:
        bad_memory.add(i)
        good_memory[i]=kart
    elif bad == -N_obj:
        break
    else:
        good_memory[i]=kart

SyntaxError: invalid syntax (2666780711.py, line 35)

In [ ]:
def get_sols2(r):
    n=int(r.readline())
    sols=[]
    for _ in range(n):
        lecture = np.array(list(map(int, r.readline().split())))
        sols.append(lecture)
    return np.array(sols)

In [ ]:
with open('solutions/final_30.txt', 'r') as r:
    pareto = get_sols2(r)

In [ ]:
from pymoo.indicators.hv import HV
ref_15 = np.array([112, 206, 247, 418])
ref_20=np.array([137, 304, 414, 531])
ref_30 = np.array([211,381,581,835])
lap = 30
if lap ==15:
    ref_point = ref_15
elif lap ==20:
    ref_point=ref_20
elif lap ==30:
    ref_point=ref_30
else:
    pt=900
    ref_point = np.array([pt, pt, pt, pt])

ind = HV(ref_point=ref_point)
hv = ind(pareto)
print("HV", hv)
print(f"Norm: {hv/(np.prod(ref_point))}")

In [ ]:
import numpy as np
def create_mat(positions, N):
    vals=[]
    tmp = np.zeros((N,N))
    for index in positions:
        tmp[index[0]][index[1]]=1
    vals.append(tmp)
    return vals

In [ ]:
def fast_compute_objectifs(coordinates, objectifs):
    return [sum([objectif[x, y] for x, y in coordinates]) for objectif in objectifs]

In [ ]:
def sol_is_valid(coordinates):
    rows = set([x for x, y in coordinates])
    columns = set([y for x, y in coordinates])
    if len(rows) == len(coordinates) and len(columns) == len(coordinates):
        # print("Il n'y a pas de doublons.")
        return True
    else:
        # print("Il y a des doublons.")
        return False

In [ ]:
import random
def fast_random_permute(tuple_list, N):
    list1, list2 = zip(*tuple_list)
    voisins = []
    for i in range(N):
        l1 = [(x, y) for x, y in zip(list1, random.sample(list2, len(list2)))]
        l2 = [(x, y) for x, y in zip(random.sample(list1, len(list1)), list2)]
        l3 = make_n_permute(tuple_list,3)
        l4 = make_n_permute(tuple_list,2)
        if sol_is_valid(l1) and l1 not in voisins:
            voisins.append(l1)
        if sol_is_valid(l2) and l2 not in voisins:
            voisins.append(l1)
        if sol_is_valid(l3) and l3 not in voisins:
            voisins.append(l1)
        if sol_is_valid(l4) and l4 not in voisins:
            voisins.append(l4)
    return voisins

In [ ]:
# coordinates = [(0, 0), (1, 1), (2, 2), (3,3)]
# matrix = np.array([[1, 0, 3, 2], [0, 6, 7, 5], [9, 0,11,0], [13,14,0,16]])
# print(matrix)
# vals = create_mat(coordinates, 4)
# sol_is_valid(coordinates)
# print(compute_objectif([matrix],vals[0], 4))
# print(fast_compute_objectifs(coordinates,matrix))

In [ ]:
# voisins = fast_random_permute(coordinates, 5)
# print(voisins)
# for voisin in voisins:
#     vals = create_mat(voisin, 4)
#     print("---------------------")
#     sol_is_valid(voisin)
#     print(compute_objectif([matrix],vals[0], 3))
#     print(fast_compute_objectifs(voisin,matrix))

In [ ]:
N_obj= 4
lap=30
objectifs, dim, sols = load_datas(f'datas/LAP-{lap}.dat', N_obj, get_sol=False)
print(len(objectifs))

In [ ]:
def generate_identity_and_reverse(dim):
    identi = np.identity(dim)
    indices = np.nonzero(identi)
    start = list(zip(indices[0], indices[1]))
    flip_start = np.fliplr(identi)
    indices = np.nonzero(flip_start)
    flip_start = list(zip(indices[0], indices[1]))
    return [start, flip_start]


In [ ]:
def generate_best_single_obj(objectifs):
    vals =[]
    objs_copy = np.copy(objectifs)
    for i in range(len(objectifs)):
        m = Munkres()
        indexes = m.compute(objs_copy[i])
        vals.append(indexes)
    return vals

In [ ]:
def make_n_permute(tuple_list,n):
    coordinates = tuple_list.copy()
    samples = random.sample(coordinates, n)
    l1, l2 = zip(*samples)
    line_permute = [(x, y) for x, y in zip(random.sample(l1, len(l1)), l2 )]
    for sample in samples:
        coordinates.remove(sample)
    return coordinates + line_permute

In [ ]:
P_temp = generate_best_single_obj(objectifs) + generate_identity_and_reverse(dim)
print(len(P_temp))
i = 1
vals=[]
for val in P_temp:
    vals+=fast_random_permute(val, 10)
    i+=1
P_temp+=vals

In [ ]:
A = [] # archive val obj
Pa=[]
for p in P_temp:
    obj_voisin = fast_compute_objectifs(p, objectifs)
    if len(A)==0:
        A.append((obj_voisin, p))
        Pa.append(p)
        pass
    A, Pa = update(A, Pa, obj_voisin, p, N_obj)
P=Pa
Pa = []

In [ ]:
import time
import numpy as np


n_voisinage_permute=10
max_time = 600
start = time.time()
stopper = 0
while P:
    i_start = time.time()
    print(f"Iterations: {stopper+1}")
    print(f"Lenght of A: {len(A)}")
    print(f"Lenght of P: {len(P)}")
    for x in P:
        voisinage = fast_random_permute(x, n_voisinage_permute)
        # print(f"Voisinage généré: {len(voisinage)}/{n_voisinage_permute*3}")
        for voisin in voisinage:
            obj_voisin = fast_compute_objectifs(voisin, objectifs)
            A, Pa = update(A, Pa, obj_voisin, voisin, N_obj)
    i_end = time.time()
    print(f"Iteration time: {i_end-i_start}")
    print("----------------------------------")
    P = Pa
    Pa = []
    if (i_end-start) >= max_time:
        break
    stopper+=1
end = time.time()
print(f"Total time: {end-start} seconds")

In [ ]:
from pymoo.indicators.hv import HV
pareto=[]
for a in A:
    pareto.append(np.array(a[0]))
pareto=np.array(pareto)
ref_15 = np.array([112, 206, 247, 418])
ref_20=np.array([137, 304, 414, 531])
ref_30 = np.array([211,381,581,835])
if lap ==15:
    ref_point = ref_15
elif lap ==20:
    ref_point=ref_20
elif lap ==30:
    ref_point=ref_30
else:
    pt=900
    ref_point = np.array([pt, pt, pt, pt])

ind = HV(ref_point=ref_point)
hv = ind(pareto)
print("HV", hv)
print(f"Norm: {hv/(np.prod(ref_point))}")

In [ ]:
with open(f'solutions_multi/sols_Lap-{lap}_{round(hv/(np.prod(ref_point))*100, 2)}.txt', 'w') as w:
    w.write(f'{len(A)}\n')
    for a in A:
        w.write(' '.join(str(p) for p in a[0]))
        w.write('\n')